In [ ]:
import torch
import matplotlib.pyplot as plt
#from datasets import concatenate_datasets

import sys
sys.path.append('./src')

from data import load_data, get_data_sl
from visualize import plot_outcome_distribution
from model import get_model
from causal import compute_ead

## Load Data

In [ ]:
supervised = load_data(environment='supervised')
unsupervised = load_data(environment='unsupervised')

### Sanity Check

In [ ]:
encoder_name = "vit"
processor, model = get_model(encoder_name)

In [ ]:
idx = 15000
img = supervised[idx]['image']
outcome = supervised[idx]['outcome']
inputs = processor(images=img, return_tensors="pt")
outputs = model(**inputs)
logits = outputs.logits
print("Top 5 predicted labels with associated probabilities:")
top_5 = torch.topk(logits, 5)
probs = logits.softmax(-1)[0][top_5.indices][0]
for i, (idx, prob) in enumerate(zip(top_5.indices[0], probs), 1):
    print(f"    {i}. {model.config.id2label[idx.item()]}: {prob.item():.2%}")

img = img.permute(1, 2, 0)
plt.title(f"Y2F: {int(outcome[0])}, B2F: {int(outcome[1])}")
plt.imshow(img);

In [ ]:
inputs = processor(images=img, return_tensors="pt")
outputs = model(**inputs, output_hidden_states=True)
outputs.hidden_states[-1][:,0].shape
# outputs.hidden_states[-1].mean(dim=[2,3]).shape

## Supervised Learning

In [ ]:
from sklearn.model_selection import train_test_split

X, y, _ = get_data_sl(environment="supervised", encoder_name="dino")
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

outcome_idx = 0

model = LogisticRegression(solver='liblinear', max_iter=10) 
model.fit(X_train, y_train[:,outcome_idx])

y_train_pred = model.predict(X_train)  
y_val_pred = model.predict(X_val)  
print(classification_report(y_train[:,outcome_idx], y_train_pred))
print(classification_report(y_val[:,outcome_idx], y_val_pred))

In [ ]:
import svmutil  
from sklearn.metrics import classification_report

outcome_idx = 0

prob = svmutil.svm_problem(y[:, otcome_idx], X)
param = svmutil.svm_parameter('-s 0 -t 2 -c 1 -g 0.1')  
model = svmutil.svm_train(prob, param) 

y_pred = model.predict(X)  
print(classification_report(y[:,outcome_idx], y_pred))

In [ ]:
import matplotlib.pyplot as plt

id_1s = (torch.Tensor(y_pred) == 1).nonzero(as_tuple=True)[0]#[0]
id_1 = id_1s[0].item()

train = load_data(environment='train')
example = train[id_1]["image"]

img = example.numpy().transpose(1, 2, 0)
plt.imshow(img)
plt.show()

## Causal Inference

In [ ]:
plot_outcome_distribution(train, save=True)

In [ ]:
rct = train #concatenate_datasets([train, test])
Y = rct["outcome"]
T = rct["treatment"]

EAD_B_y, EAD_inf_y = compute_ead(Y, T, color="Yellow")   
EAD_B_b, EAD_inf_b = compute_ead(Y, T, color="Blue")   